In [1]:
import cv2
import pytesseract
import re

# Aadhaar + DOB detection from image
img = cv2.imread("WhatsApp Image 2025-06-14 at 10.54.00 AM.jpeg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Set tesseract path
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# OCR data
dic = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
n = len(dic['text'])

# Pattern for DOB
dob_pattern = r'\b\d{1,2}[\/\-\.]\d{1,2}[\/\-\.]\d{2,4}\b'

for i in range(n):
    word = dic['text'][i]
    
    # 🔹 Detect DOB
    if re.fullmatch(dob_pattern, word):
        x, y, w, h = dic['left'][i], dic['top'][i], dic['width'][i], dic['height'][i]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 🔹 Detect Aadhaar from 3 consecutive 4-digit parts
    if i+2 < n:
        text1 = dic['text'][i].strip()
        text2 = dic['text'][i+1].strip()
        text3 = dic['text'][i+2].strip()

        if re.fullmatch(r'\d{4}', text1) and re.fullmatch(r'\d{4}', text2) and re.fullmatch(r'\d{4}', text3):
            x = min(dic['left'][i], dic['left'][i+1], dic['left'][i+2])
            y = min(dic['top'][i], dic['top'][i+1], dic['top'][i+2])
            w = max(dic['left'][i] + dic['width'][i],
                    dic['left'][i+1] + dic['width'][i+1],
                    dic['left'][i+2] + dic['width'][i+2]) - x
            h = max(dic['top'][i], dic['top'][i+1], dic['top'][i+2]) - y + dic['height'][i]
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

# Show output
cv2.namedWindow("image1", cv2.WINDOW_NORMAL)
cv2.resizeWindow("image1", 1000, 700)
cv2.imshow("image1", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
